In [2]:
#Where search engine is made
#Exploring the data
#!wc-l is a command line utility that will count the number of lines in this goodread books file
#Result should show metadata for about 7.5 million books

!wc -l goodreads_books.json.gz

3398473 goodreads_books.json.gz


In [3]:
#This command shows the size of the file (About 1.9 GB) and it shows what file type it is
!ls -lh | grep goodreads_books.json.gz

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
#Used streaming-fashion technique here--> Instead of reading whole file into memory at once, file will be read line by line (only enough memory to store a single line at a time therefore less memory used)

import gzip
# Stream the file without unzipping
with gzip.open("goodreads_books.json.gz") as f:
    line = f.readline()  #Read single line from file

In [4]:
line  #single line

b'{"isbn": "0312853122", "text_reviews_count": "1", "series": [], "country_code": "US", "language_code": "", "popular_shelves": [{"count": "3", "name": "to-read"}, {"count": "1", "name": "p"}, {"count": "1", "name": "collection"}, {"count": "1", "name": "w-c-fields"}, {"count": "1", "name": "biography"}], "asin": "", "is_ebook": "false", "average_rating": "4.00", "kindle_asin": "", "similar_books": [], "description": "", "format": "Paperback", "link": "https://www.goodreads.com/book/show/5333265-w-c-fields", "authors": [{"author_id": "604031", "role": ""}], "publisher": "St. Martin\'s Press", "num_pages": "256", "publication_day": "1", "isbn13": "9780312853129", "publication_month": "9", "edition_information": "", "publication_year": "1984", "url": "https://www.goodreads.com/book/show/5333265-w-c-fields", "image_url": "https://images.gr-assets.com/books/1310220028m/5333265.jpg", "book_id": "5333265", "ratings_count": "3", "work_id": "5400751", "title": "W.C. Fields: A Life on Film", "t

In [5]:
#Use json module to load the single line
import json

data = json.loads(line) #loading as json object. Will turn result into a python dictionary where each of the individual properties can be accessed
data

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'series': [],
 'country_code': 'US',
 'language_code': '',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
  {'count': '1', 'name': 'p'},
  {'count': '1', 'name': 'collection'},
  {'count': '1', 'name': 'w-c-fields'},
  {'count': '1', 'name': 'biography'}],
 'asin': '',
 'is_ebook': 'false',
 'average_rating': '4.00',
 'kindle_asin': '',
 'similar_books': [],
 'description': '',
 'format': 'Paperback',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'authors': [{'author_id': '604031', 'role': ''}],
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_day': '1',
 'isbn13': '9780312853129',
 'publication_month': '9',
 'edition_information': '',
 'publication_year': '1984',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'book_id': '5333265',
 'ratings_count': '3',
 'work_id': '5400751',
 'title': '

In [ ]:

def parse_fields(line): #Takes a single line and only returns the fields below (book id, title, ratings etc)
    data = json.loads(line)
    return {
        "book_id": data["book_id"],
        "title": data["title_without_series"],
        "ratings": data["ratings_count"],
        "url": data["url"],
        "cover_image": data["image_url"] #helps see the book and click the link for it
    }

In [ ]:
books_titles = [] #Read every single line in the file by using a loop
with gzip.open("goodreads_books.json.gz") as f:
    while True:
        line = f.readline()
        if not line:  #When it reaches the end of the file it will break
            break
        fields = parse_fields(line) #runs parsefields function to parse line and return dictionary
        try:
            ratings = int(fields["ratings"]) #only want to take books that have a certain #of ratings. When they don't have a lot of ratings, it's very unlikely that users have read them
        except ValueError:
            continue
        if ratings > 5: #Will only take books that have more than 5 ratings
            books_titles.append(fields)

In [ ]:
#Making dataframe
#Book titles will be a list of dictionaries that will be turned into a row in the dataframe
import pandas as pd
titles = pd.DataFrame.from_dict(books_titles)

In [ ]:
titles["ratings"] = pd.to_numeric(titles["ratings"]) #Turns ratings into a numerical column. Will let us do ratings on the comparisons column

In [ ]:
#mod title minimizes search space and #of potential characters in each title--> For efficiency
#1st thing to do is replace each character that doesn't fall within a certain set
#A regular expression that basically says anything that doesn't fall within a set of characters should be replaced with "nothing" (get rid of it)
titles["mod_title"] = titles["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)

In [ ]:
# makes all titles lowercase
titles["mod_title"] = titles["mod_title"].str.lower()

In [ ]:
#Remove any sequential spaces and replace with a single space
titles["mod_title"] = titles["mod_title"].str.replace("\s+", " ", regex=True)

In [ ]:
#Remove any title that has less than 0 characters
titles = titles[titles["mod_title"].str.len() > 0]

In [ ]:
#
titles.to_json("books_titles.json")

In [ ]:
# Returns titles
titles

In [ ]:
#Using scikit learn to build TF*IDF matrix
#Vectorizer takes a list of strings and turns it into tf*idf matrix

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

#run it on mod titles
tfidf = vectorizer.fit_transform(titles["mod_title"])

In [ ]:
#Turn a search query into a vector, and then match it up against the matrix and compare them
#To do our comparison of the vectors we'll use a metric called cosine-similarity

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re #Python regular expression library

def make_clickable(val): #Pandas method to style columns with html
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val) #Building html element for each column entry

def show_image(val): #Returns image of book using pandas
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

def search(query,vectorizer):
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())  #Same procesing as mod title but with search query
    query_vec = vectorizer.transform([query]) #turn query into vector using vectorizer
    similarity = cosine_similarity(query_vec, tfidf).flatten() #Find similarities using cosine similarity function, and search tf*idf matrix for your search query
    indices = np.argpartition(similarity, -10)[-10:] #Use numpy partition function to find the 10 most similar results. Want to find indices as well in order to index titles
    results = titles.iloc[indices] #Gives us book title rows with most similar results to query
    results = results.sort_values("ratings", ascending=False) #Sort the values on the number of ratings using pandas sort value method. Will help limit duplicate results by taking the row with the highest number of ratings

    return results.head(5).style.format({'url': make_clickable, 'cover_image': show_image}) #Returns top 5 results and styles them using html. Allows users to click on url of results, and to view the cover image.

In [5]:
search("Harry potter and the deathly hallows", vectorizer) #Return search

NameError: name 'search' is not defined

In [8]:
#Take query results and use book id's to create liked books list.
#Store as strings
liked_books=["3314", "31147619", "29983711", "9401317","9317691", "8153988", "20494944"]  #I'd generally recommend to make your liked books list list less than 10 but more than 3 books just for efficiencie's sake.